In [8]:
%pip install yfinance
import yfinance as yf
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
!pip install voila



In [2]:
import pandas as pd
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Function to create a custom income statement
def create_custom_income_statement(ticker):
    stock = yf.Ticker(ticker)
    company_name = stock.info.get('longName', 'Company Name Not Available')
    financials = stock.financials
    fiscal_year_end = financials.columns[0].strftime('%Y-%m-%d')

    total_revenue = financials.loc['Total Revenue'].iloc[0] / 1_000_000
    total_expenses = financials.loc['Total Expenses'].iloc[0] / 1_000_000 if 'Total Expenses' in financials.index else None
    ebit = financials.loc['EBIT'].iloc[0] / 1_000_000
    interest_expense = financials.loc['Interest Expense'].iloc[0] / 1_000_000 if 'Interest Expense' in financials.index else None
    tax_expense = financials.loc['Tax Provision'].iloc[0] / 1_000_000 if 'Tax Provision' in financials.index else None
    net_income = financials.loc['Net Income From Continuing Operation Net Minority Interest'].iloc[0] / 1_000_000
    other = financials.loc['Other Income Expense'].iloc[0] / 1_000_000

    ebit_margin = (ebit / total_revenue) * 100 if total_revenue != 0 else None
    pre_tax_income = ebit - interest_expense if interest_expense is not None else ebit
    effective_tax_rate = (tax_expense / pre_tax_income) * 100 if pre_tax_income != 0 else None
    share_count = stock.info['impliedSharesOutstanding'] / 1_000_000
    eps_adjusted = round(net_income / share_count, 2)

    data = {
        "Data Below is for Fiscal Year Ending:": [
            "Total Revenue (as reported)",
            "Total Expenses",
            "EBIT (adjusted)",
            "EBIT margin (adjusted)",
            "Interest Expense",
            "Other",
            "Pre-tax Income (adjusted)",
            "Tax Expense",
            "Effective Tax Rate",
            "Net Income (adjusted)",
            "EPS adjusted",
            "Fully Diluted share count"
        ],
        fiscal_year_end: [
            total_revenue,
            total_expenses,
            ebit,
            f"{ebit_margin:.2f}%" if ebit_margin is not None else None,
            interest_expense,
            other,
            pre_tax_income,
            tax_expense,
            f"{effective_tax_rate:.2f}%" if effective_tax_rate is not None else None,
            net_income,
            eps_adjusted,
            share_count
        ]
    }

    df = pd.DataFrame(data)
    return company_name, df, total_revenue, total_expenses, ebit_margin, eps_adjusted, net_income, effective_tax_rate

# Function to create materiality threshold table
def create_materiality_table(materiality_threshold, eps_adjusted, net_income, effective_tax_rate, total_revenue, total_expenses, ebit_margin):
    eps_change = materiality_threshold * eps_adjusted
    ni_change = materiality_threshold * net_income
    magic_number = ni_change / (1 - effective_tax_rate / 100) if effective_tax_rate is not None else None
    pricing_change = magic_number / total_revenue if total_revenue != 0 else None
    expense_change = magic_number / total_expenses if total_expenses != 0 else None
    volume_change = magic_number / (ebit_margin / 100) if ebit_margin != 0 else None
    revenue_percentage = volume_change / total_revenue if total_revenue != 0 else None

    materiality_data = {
        "Materiality Thresholds": [
            f"{materiality_threshold * 100:.0f}% change to EPS",
            "Materiality Threshold as measured in NI",
            "(Magic Number) as measured in pre-tax EBIT",
            "As measured in changes to sales PRICING",
            "As measured in changes to EXPENSES",
            "As measured in changes to sales VOLUME",
            "(Sales Volume) As a percentage of Revenue"
        ],
        "Value": [
            f"$ {eps_change:.2f}",
             f"$ {ni_change:.0f}",
            f"$ {magic_number:.0f}" if magic_number is not None else None,
             f"{pricing_change:.2%}" if pricing_change is not None else None,
             f"{expense_change:.2%}" if expense_change is not None else None,
           f"$ {volume_change:.0f}" if volume_change is not None else None,
            f"{revenue_percentage:.2%}" if revenue_percentage is not None else None
        ]
    }

    materiality_df = pd.DataFrame(materiality_data)
    return materiality_df

# Create an input box for ticker
ticker_input = widgets.Text(
    description='Enter Ticker:',
    value='AAPL'  # Default value
)

# Create an input for Materiality Threshold
materiality_input = widgets.FloatSlider(
    value=0.10,  # Default 10%
    min=0,
    max=0.3,
    step=0.01,
    description='Materiality Threshold (%):',
    style={'description_width': 'initial'},
    readout_format='.2%'
)

output_area = widgets.Output()

def update_income_statement(ticker, materiality_threshold):
    with output_area:
        clear_output(wait=True)
        try:
            company_name, income_statement, total_revenue, total_expenses, ebit_margin, eps_adjusted, net_income, effective_tax_rate = create_custom_income_statement(ticker)

            display(HTML(f"<h3 style='text-align:left; font-weight:bold;'>{company_name}</h3>"))
            display(income_statement)

            materiality_table = create_materiality_table(materiality_threshold, eps_adjusted, net_income, effective_tax_rate, total_revenue, total_expenses, ebit_margin)
            display(HTML("<h4 style='text-align:left; font-weight:bold;'>Materiality Calculations</h4>"))
            display(materiality_table)

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

# Display the input box, materiality slider, and output area
display(ticker_input, materiality_input, output_area)

def on_change(change):
    update_income_statement(ticker_input.value, materiality_input.value)

ticker_input.observe(on_change, names='value')
materiality_input.observe(on_change, names='value')

# Initially display the income statement and materiality table for the default value
update_income_statement(ticker_input.value, materiality_input.value)


Text(value='AAPL', description='Enter Ticker:')

FloatSlider(value=0.1, description='Materiality Threshold (%):', max=0.3, readout_format='.2%', step=0.01, sty…

Output()